In [1]:
# load data
import pandas as pd

counts=pd.read_csv("/mnt/input_data/counts.csv")
metadata=pd.read_csv("/mnt/input_data/meta.csv")

# subset expression
expression_table=counts.iloc[:, 1:17]

print(counts.head())
print(metadata.head())

           gene          P1T          P6T         P14T         P15T  \
0        GIMAP2   820.708155  1351.244295  1313.127405   986.841996   
1        FAM78A   938.414983  1771.936259  2060.075202  1294.263123   
2         SP110   667.365316   645.899346   873.144457   669.330461   
3  RP11-54O7.17    21.597583     9.145477     1.705360     6.054245   
4          IL16  2204.033348  4118.322820  4398.976808  3280.055606   

          P16T     Cont.19      Cont.21      Cont.32     Cont.51     Cont.54  \
0  1395.368089  377.071125   411.469285   327.364017  345.104563  507.185670   
1  1762.652828  358.788889   436.284403   617.186289  498.361862  251.768426   
2   974.717192  258.236589   309.323334   290.573107  242.565508  262.714879   
3     1.569593   26.280715    79.639216   631.452152  389.207383  547.322665   
4  4145.294853  639.878273  1236.716239  1484.400599  670.362858  614.825794   

       Cont.70     Cont.78          P3T         P10T         P13T     CONT.34  
0   401.9351

In [2]:
# prepare data for splitting into train and test sets
from sklearn.model_selection import train_test_split

X=expression_table.T
y=metadata["treatment"]  # Labels

# split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=716) # 70% training and 30% test

print("train", list(X_train.index))
print("test", list(X_test.index))

train ['P16T', 'Cont.54', 'Cont.32', 'P10T', 'Cont.70', 'Cont.19', 'Cont.51', 'P1T', 'P15T', 'Cont.78', 'Cont.21']
test ['P6T', 'P3T', 'CONT.34', 'P13T', 'P14T']


In [3]:
# import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

# create a classifier
clf=RandomForestClassifier(n_estimators=100, random_state=716)

# train the model
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

# import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# model Accuracy, how often is the classifier correct? 100% possibly due to small size?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 1.0


In [4]:
# get feature importance, only 98 features have any predictive value
feature_imp = pd.Series(clf.feature_importances_,index=counts.gene).sort_values(ascending=False)

print(feature_imp.head(5))
print(feature_imp.tail(5))

# export to csv
feature_imp.to_csv("/mnt/output_data/random_forest_feature_importance.csv", index=True)

gene
VRK2       0.020202
A1CF       0.010101
GDPD1      0.010101
DZIP1      0.010101
MYCBPAP    0.010101
dtype: float64
gene
CKAP4        0.0
FGD5P1       0.0
TM9SF1       0.0
STX17-AS1    0.0
SNAP23       0.0
dtype: float64


In [5]:
# continue filtering feature list until all are important or 5 rounds max
counter=0

while counter < 10 or (feature_imp == 0).any():
    # advance counter
    counter += 1  # Increment the counter
    print(counter)
    
    # filter genes below importance limit, increase threshold with each round 
    kept_features=feature_imp[feature_imp > 0.01*counter].index

    counts_kept=counts[counts.gene.isin(kept_features)]
    expression_table_kept=counts_kept.iloc[:, 1:17]

    X=expression_table_kept.T
    y=metadata["treatment"]  # Labels

    # split dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=716) # 70% training and 30% test

    print("train", list(X_train.index))
    print("test", list(X_test.index))

    # import Random Forest Model
    from sklearn.ensemble import RandomForestClassifier

    # create a classifier
    clf=RandomForestClassifier(n_estimators=100, random_state=716)

    # train the model
    clf.fit(X_train,y_train)

    y_pred=clf.predict(X_test)

    # import scikit-learn metrics module for accuracy calculation
    from sklearn import metrics
    # model Accuracy, how often is the classifier correct? 100% possibly due to small size?
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

    # get feature importance, only 98 features have any predictive value
    feature_imp = pd.Series(clf.feature_importances_,index=counts_kept.gene).sort_values(ascending=False)

    print(feature_imp.head(3))
    print(feature_imp.tail(3))
    print(feature_imp.shape)
    
    # export to csv
    feature_imp.to_csv("".join(["/mnt/output_data/round_", f'{counter}', "_filtered_random_forest_feature_importance.csv"]), index=True)

1
train ['P16T', 'Cont.54', 'Cont.32', 'P10T', 'Cont.70', 'Cont.19', 'Cont.51', 'P1T', 'P15T', 'Cont.78', 'Cont.21']
test ['P6T', 'P3T', 'CONT.34', 'P13T', 'P14T']
Accuracy: 1.0
gene
RP11-34F20.7    0.050505
SGCE            0.050505
TTC23           0.040404
dtype: float64
gene
A1CF     0.0
SSR1     0.0
CCL23    0.0
dtype: float64
(98,)
2
train ['P16T', 'Cont.54', 'Cont.32', 'P10T', 'Cont.70', 'Cont.19', 'Cont.51', 'P1T', 'P15T', 'Cont.78', 'Cont.21']
test ['P6T', 'P3T', 'CONT.34', 'P13T', 'P14T']
Accuracy: 1.0
gene
CASP1      0.111111
MORF4L2    0.090909
ASB9       0.070707
dtype: float64
gene
ABHD3           0.010101
ALDH7A1P1       0.010101
RP11-440G5.2    0.010101
dtype: float64
(26,)
3
train ['P16T', 'Cont.54', 'Cont.32', 'P10T', 'Cont.70', 'Cont.19', 'Cont.51', 'P1T', 'P15T', 'Cont.78', 'Cont.21']
test ['P6T', 'P3T', 'CONT.34', 'P13T', 'P14T']
Accuracy: 1.0
gene
RINL     0.101010
TTC23    0.090909
CTC1     0.090909
dtype: float64
gene
VRK2    0.030303
GCKR    0.030303
SGCE    0.02